# 经验之谈 02: 三种深度学习人脸检测器性能的简单比较

让一个小朋友去给照片上的人脸打上马赛克，小朋友完成这个任务大概可以分为两步：1.找到人脸在图像中的位置，2.对该位置进行涂鸦。这个小例子中最基础的工作就是找到人脸的位置，也就是业内通常所说的“人脸检测”。仔细思考一下我们日常生活中与人脸有关的事情就会发现，无论任务多么复杂，总是需要先找到人脸的位置。人工智能就如同是一个智商很低，运算速度很快的小朋友，同样的道理，它在完成人脸验证、人脸识别、人脸美化等等任务时，总是需要使用检测器获取人脸位置。因此，人脸检测是与人脸有关的人工智能技术的基础。

## 1. 什么是人脸检测？

虽然上文中对人脸检测已经有了一些语言上的描述，但是对于一些刚接触计算机视觉或者人脸相关技术的朋友而言，可能仍然是一头雾水。下面我们通过照片实例来说明人脸检测究竟是什么。

![title](images/face_detection_comp/retina_face/t1.jpg)

上图是一部美剧中几个主角的照片，现在我们想要知道这张照片中到底有几个人。当然，作为人类可以很直接地看出这里包含了六个人，那么计算机是否能完成这个任务呢？下面就展示一下计算机处理该图片之后得到的结果。

![title](images/face_detection_comp/retina_face/detector_t1_test.jpg)

从计算机输出的这种图像可以看出，算法程序将图像的每个人的人脸都用红色的方框标志出来了。计算机只需要数一下得到的红框数就可以知道这里面有多少个人脸。此外，计算机还可以继续分析每个人的表情、性别、年龄以及头部的注视方向等等。从这个例子中就很直观地表明了人脸检测所做的事情，就是找出图像中人脸的位置。

## 2. 深度学习人脸检测算法

在深度学习兴起之前，人脸检测的算法逻辑是“滑动窗口”+“人脸分类器”，也就是使用一个固定大小的窗口在图像上按照固定的步幅滑动，每次得到一个新的局部图像就使用分类器来进行判断是否包含人脸。等到一个图像上所有的窗口都判断完成，就知道这张图像上是否存在人脸，以及如果存在人脸的话人脸在哪里。了解了这个算法逻辑是不是觉得传统方法的人工智能其实很傻，只是运算速度快显得很厉害的样子。

深度学习兴起之后，这个过程封装在一个黑箱之中，大家只关心丢进去一张图片就可以拿到图像中所有人脸的位置。最近几年，关于目标检测的深度学习算法层出不穷，人脸检测的在目标检测技术的驱动下也得到很不错的发展。这几天在github上发现了三个人脸检测的repo，正好代表了目前风头正劲的几种算法。这里就对这三个深度学习的人脸检测器进行一下简单的比较。由于目前还在尝试重新编写训练模型的代码，因此本文暂时先不涉及算法原理的深入探讨，之后训练效果符合预期再进行相应的精细分析。

首先出场的是SSD(Single Shot MultiBox Detector)算法[1]。该算法在2015年提出，一个使用全卷积层的神经网络来进行目标检测，由于在多个著名数据集上取得非凡精度而名声大噪。在github上yeephycho发布了一个使用mobilenet+SSD的repo，我们就使用这个代码来测试一下。我们选择一张目前为止，人类智商密度最高的照片作为评测的输入图像，如下所示：

![title](images/face_detection_comp/mobilenet_ssd/sample.jpg)

mobilenet+SSD算法的处理结果如下图所示：

![title](images/face_detection_comp/mobilenet_ssd/out_sample.jpg)

从图中可以看出，算法程序找出了图像中的绝大多数人脸，并且所标志的人脸位置也都是非常准确的。只是这些遗漏的人脸在我们人类的眼睛看来似乎是不可接受的错误。那么有没有更好的算法呢？下文将揭晓答案。

接下来出场的是YOLOv3算法[2]，该算法继承YOLO(You Only Look Once)算法。YOLO算法是在一个并不太主流的框架上实现的，不过它从一开始就用其效果和速度惊艳了学术界。其所依赖的darknet框架是完全用C语言写成的，这些对于一些python语言的框架更具有效率的优势。神经网络的结构也并不是采用SSD那种全卷积的结构，具体的细节大家可以在论文中查看。这里就使用github上的sthanhng提供的yoloface代码直接展示它在人脸检测任务上的结果。

![title](images/face_detection_comp/yolov3/sample_yoloface.jpg)

从以上结果来看，yolov3比mobilenet+SSD的效果要好很多，至少是没有出现漏掉的人脸。但是仔细查看会发现，算法程序给出的人脸位置并不是太准确，这对于一些定位精度要求高的任务而言是不合适的。

接下来出场的是RetinaFace算法[3]，这款算法在训练时采用了更多的信息，比如五个面部的轮廓点。或许有些朋友会认为这种比较不公平，不过我倒认为这并不是重点。毕竟我们的目的是寻找更优化的解决方案，而不是在某种规则下拼个你死我活。这款算法使用了最近几年比较热门的“图卷积”，而且在通常检测任务的损失函数上添加了两项，一项是五个人脸轮廓点的回归损失，一个是局部图像生成损失。从结果来看，这两个损失函数对于人脸检测的精度提升大有裨益。闲言少叙，这里使用github上的deepinsight共享的insightface代码直接给出它在人脸检测任务上的结果。

![title](images/face_detection_comp/retina_face/detector_sample_test.jpg)

从上图可以看出，RetinaFace所给出的人脸位置比YOLOv3的结果要精确很多，而且还给出了5个面部关键部位的点坐标。上图标志五个面部点使得图像看起来稍显诡异，但想必这些聪明的大脑对此并不介意。从目前收集到的信息来看，RetinaFace算法是目前人脸检测方向上精度最好的算法。

## 3. 精度和速度

在实际使用过程中，精度当然是我们不懈追求的，但速度也是一个值得考虑的重要标准。这里就使用一个分辨率为1024x682、多达100多个人脸的图像测试一下这三种算法的精度和速度。这里的精度都是使用肉眼观察，更加合理的做法是使用数据集评测。不过鉴于这种数据集上的测试数据在原论文中已经非常详实，这里就不再画蛇添足了。代码还是使用上文中提到的三个repo。硬件平台是MacbookPro 2018，CPU为主频2.3G的8代I5芯片，内存大小为16GB。

我们首先看看mobilenet+SSD算法程序给出的效果图像：

![title](images/face_detection_comp/mobilenet_ssd/out_t3.jpg)

这张结果图显示mobilenet+SSD的检测率非常低，但是不得不澄清，这并不是完全是算法的问题。因为图像中每个人脸的分辨率都很低，而在训练算法模型时对应的尺度设置没有覆盖这种小分辨率的人脸，将会出现这种状况。因此，以SSD算法本身的能力，在训练参数设置合理的情况下，应该可以得到更好的效果。

接下来我们来看看YOLOv3在这种图像输入下给出的结果：

![title](images/face_detection_comp/yolov3/meeting_11_304_yoloface.jpg)

从YOLOv3给出的这张结果图来看，算法表现确实非常不错，有个别的漏检和误检情况。人脸位置的精度与上文说到的一样都是差强人意。这里可以先剧透一下，这种算法这精度可接受的情况下，速度很不错，也是一个值得使用的理由。

最后，我们来看看RetinaFace算法给出的结果：

![title](images/face_detection_comp/retina_face/detector_t3_test.jpg)

上图展示了RetinaFace算法检测到了102张人脸，目力所及之处还未发现漏检和误检，并且所给出的人脸位置精度很不错。在这种人脸如此稠密、单个人脸分辨率如此低的情况下能给出这样的检测结果，确实让人眼前一亮。

除了检测率和定位精度上的结果差异之外，运行速度的差异也是一个关注点，下面就展示这三种算法的计算速度。其中mobilenet+SSD是使用tensorflow框架，YOLOv3是使用OpenCV接口调用，RetinaFace是使用mxnet框架，这三者都在底层有大量速度优化。因此虽然底层有所不同，但也算是各尽全力，其比较也具有一定的参考意义。

|算法名|100帧运行时间(秒)|FPS|
|:-----:|:--:|:--:|
|mobilenet+SSD|8.997|11.11|
|YOLOv3|28.53|3.51|
|RetinaFace|267.72|0.37|

从速度的比较上来看，RetinaFace虽然精度高，但是速度却比较慢。因此在选择模型时需要进行权衡，有些应用对精度非常敏感，而速度可以适当牺牲；有些任务对实时性要求极高，精度反而可以稍低。

到此，关于三种深度学习人脸检测器性能的简单比较就到这里。对人脸检测器感兴趣的朋友是不是已经跃跃欲试了，这三款模型都能在github上下载源码。不过其对应的训练代码通常并不包含，或者训练所需软硬件条件一时难以齐备。之后我有空训练出符合预期的模型再来讨论每一种模型中更多的细节。本文的notebook版本在github上的cnbluegeek/notebook仓库中共享，欢迎感兴趣的朋友前往下载。

## 参考文献：

[1] Liu, Wei and Anguelov, Dragomir and Erhan, Dumitru et.al. SSD: Single Shot MultiBox Detector. arXiv preprint arXiv:1512.02325, 2015

[2] Farhadi, Joseph Redmon Ali. YOLOv3: An Incremental Improvement. 2017

[3] Deng, Jiankang and Guo, Jia and Zhou, Yuxiang et.al. RetinaFace: Single-stage Dense Face Localisation in the Wild. arXiv:1905.00641 [cs], 2019